import ee
import geemap
geemap.set_proxy(port = 56940)
ee.Initialize()

In [8]:
roi = ee.Geometry.Polygon(  [   [180, 90],
                                [180, -90],
                                [-180, 90],
                                [-180 , -90]
                            ],None,False,)
start = ee.Date('2014-06-01')
finish = ee.Date('2014-10-01')
filteredCollection = ee.ImageCollection('LANDSAT/LC08/C01/T1') \
    .filterBounds(roi) \
    .filterDate(start, finish) \
    .filter(ee.Filter.lt("CLOUD_COVER",10))

mosaic = filteredCollection.mosaic()

In [15]:
Map = geemap.Map(center=[40, -100], zoom=4)
vizParams = {'bands': ['B5', 'B4', 'B3'],
             'min': 5000, 'max': 15000, 'gamma': 1.3}
Map.addLayer(mosaic, vizParams, 'Landsat 8 image',0)


In [16]:
first = mosaic
wi = first.expression('(1.5*g-r-1*nir)**2/(g+nir)**2',{
    'b':first.select('B2'),
  'g':first.select('B3'),
  'r':first.select('B4'),
  'nir':first.select('B5'),
  'swir1':first.select('B6'),
  'swir2':first.select('B7')
})
vizParams2 = {'min': 0, 'max': 1, 'gamma': 1.3}
Map.addLayer(wi, vizParams2, 'wi')

palette = ['42C8F0']
wi_mask = wi.updateMask(wi.lt(0.05))
Map.addLayer(wi_mask, {'min': 0, 'max': 1,'palette':palette}, 'wi_mask')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…